# Root Finding: Secant Method
---

GENERAL PROBLEM: find the real roots of a given function $f(x)$ in the case when closed-form solutions are not available. That is, find the values of $x$ that satisfy the equation

\begin{align}
  f(x) = 0
\end{align}

where $x$ is a real variable, and $f(x)$ is some non-linear function.

IDEA: use the roots of secant lines to construct successive approximations to the actual root of the function. 

PRE-REQUISITES:
- Bisection method

REFERENCES:
- [1] DeVries and Hasbun, *A First Course in Computational Physics, 2nd edition*.
- [2] Burden and Faires, *Numerical Analysis, 7th edition*.
- [3] Ralston and Rabinowitz, *A First Course in Numerical Analysis, 2nd edition*.
- [4] Press et al, *Numerical Recipes: the Art of Scientific Computing, 3rd edition*.

## 1. Summary of the method

As with the bisection method, we assume that we have identified an interval $[a,b]$ that brackets a root of $f(x)$.

The idea behind the secant method is to connect the points $(a,f(a))$ and $(b,f(b))$ by a line, then use the root of that line as an improved approximation of the root of $f(x)$, and repeat. At each iteration, the previous two estimates are used to construct the next secant line. The process is continued until the root is located to within some given tolerance, or some maximum allowed number of iterations is reached.
 
The equation for the line that passes through points $(x_{n-2}, f(x_{n-2}))$ and $(x_{n-1}, f(x_{n-1}))$ is

\begin{align}
  y(x) = \frac{(f(x_{n-1}) - f(x_{n-2}))}{(x_{n-1} - x_{n-2})}(x - x_{n-1}) + f(x_{n-1})
\end{align}

The root of this equation is taken as the next approximation to the root of $f(x)$. This yields the iteration equation 

\begin{align}
  x_{n} = x_{n-1} - f(x_{n-1})\frac{x_{n-1} - x_{n-2}}{f(x_{n-1}) - f(x_{n-2})}
\end{align}

The relative uncertainty in the location of the root at the end of an iteration is taken to be

\begin{align}
  \mathsf{REL}
  = \left|\frac{\text{(current estimate)} - \text{(previous estimate)}}{\text{(current estmate)}}\right|
  = \left|\frac{x_{n} - x_{n-1}}{x_{n}}\right|
\end{align}

We should also calculate the error of any proposed root candidate  

\begin{align}
  \mathsf{ABS} = \left|f(x_{n})\right|
\end{align}

The process of searching for a root should continue until both of these errors are less than some specified tolerance (or the maximum allowed number of iterations is reached).

(PROGRAMMER'S NOTE: when starting from values close to actual root, the secant method often converges faster than the bisection method. However, one drawback is that the secant method does not always keep the root bracketed, and so unlike the bisection method, convergence is not assured. It is therefore advisable to issue a warning any time an iteration occurs when the root is no longer bracketed.)

## 2. Algorithm

**INPUT**
- an interval $[a,b]$ where a root of the function in question is known to exist.
- TOL, the relative error tolerance that the answer is required to have.
- $i_\mathrm{max}$, maximum number of iterations allowed.

**Initialize loop**
- set $x_{-1}=a$ and $x_{0}=b$
- set $i = 1$

**Loop** while $i \leq i_\mathrm{max}$

- calculate root of secant line
  - $x_{i} = x_{i-1} - f(x_{i-1})(x_{i-1} - x_{i-2})/(f(x_{i-1}) - f(x_{i-2}))$


- calculate the relative uncertainty using:  REL $= \left|(x_{i} - x_{i-1})/x_{i}\right|$


- calculate ABS $= |f(x_{i})|$


- if (REL $\leq$ TOL) and (ABS $\leq$ TOL), stop. Otherwise, continue.


- if $i > i_\mathrm{max}$, stop. Otherwise, go back to the top of the loop.


- i = i + 1

## 3. CODE

In [18]:
%%writefile secant.py

import numpy as np
import sys

# secant method
def secant(f, x0, x1, TOL, imax):
    """
    searches for roots of f(x) using the secant method
    
    INPUT:
    f = function whose roots are being sought 
    x0 = initial guess 1
    x1 = initial guess 2
    TOL = allowed tolerance
    imax = maximum number of iterations
    
    OUTPUT:
    location of the root to within the allowed tolerance, or failure message
    """
    
    # test initial bracket 
    if np.sign(f(x0))*np.sign(f(x1)) > 0:
        # if sgn > 0, problem may not be well-defined
        print('WARNING: function has the same sign at both ends of the interval.')
        print('This may lead to convergence problems. Proceed with caution!')

    # initialize iteration
    xOld = x0
    xNew = x1
    
    # iterate search using bisection method
    i = 1  # reset iteration number
    while i <= imax:
            
        # rotate previous approximate locations of root
        xOldOld = xOld # xOld --> xOldOld
        xOld = xNew    # xNew --> xOld
        
        # update approximate location of root
        fOld = f(xOld)
        xNew = xOld - fOld*(xOld - xOldOld)/(f(xOld) - f(xOldOld))
        print('iteration',i,': approximate location of root at',xNew)
    
        # calculate errors
        xErr = np.abs((xNew - xOld)/xNew)
        fErr = f(xNew)

        # check if errors are within the allowed tolerance
        if (xErr <= TOL and fErr <= TOL):
            best = xNew #best estimate
            delta = np.abs(xNew-xOld) #uncertainty
            print('SUCCESS! Root has been located to within the specified tolerance after',i,'iterations.')
            print('Root is located at',best,'+/-',delta)
            return

        # print message if max iteration has been reached
        if i == imax:
            print('FAIL! Max number of iterations has been reached. Stopping.')
            return
        
        # increment iteration number
        i = i + 1

Overwriting secant.py


In [19]:
%run secant.py

## 4. Example

In [20]:
# define function whose roots we are searching for
def myfunc(x):
    return np.cos(x) - x

In [21]:
secant(myfunc, 0., 1., 1e-9, 100)

iteration 1 : approximate location of root at 0.6850733573260451
iteration 2 : approximate location of root at 0.736298997613654
iteration 3 : approximate location of root at 0.7391193619116293
iteration 4 : approximate location of root at 0.7390851121274639
iteration 5 : approximate location of root at 0.7390851332150012
iteration 6 : approximate location of root at 0.7390851332151607
SUCCESS! Root has been located to within the specified tolerance after 6 iterations.
Root is located at 0.7390851332151607 +/- 1.5942802633617248e-13


## 5. Another example

In [22]:
# define function whose roots we are searching for
def myfunc2(x):
    return np.cos(x) - x*np.sin(x)

In [23]:
secant(myfunc2, 0., 2., 1e-9, 100)

iteration 1 : approximate location of root at 0.6182873909407172
iteration 2 : approximate location of root at 0.8526543004751521
iteration 3 : approximate location of root at 0.8611192293078886
iteration 4 : approximate location of root at 0.8603316786475832
iteration 5 : approximate location of root at 0.8603335885486075
iteration 6 : approximate location of root at 0.8603335890193801
SUCCESS! Root has been located to within the specified tolerance after 6 iterations.
Root is located at 0.8603335890193801 +/- 4.707725320685086e-10


## 6. Bracket hunting add-on
(see notebook on Bisection method.)

In [24]:
%run getbrackets.py

In [25]:
brackets = get_brackets(myfunc2, 0, 10, 1000)

sign change detected between 0.8508508508508509 and 0.8608608608608609
sign change detected between 3.4234234234234235 and 3.4334334334334335
sign change detected between 6.436436436436437 and 6.446446446446447
sign change detected between 9.51951951951952 and 9.52952952952953


In [26]:
nr = 1
secant(myfunc2, brackets[nr][0], brackets[nr][1], 1e-9, 100)

iteration 1 : approximate location of root at 3.4256142054221073
iteration 2 : approximate location of root at 3.425618451255579
iteration 3 : approximate location of root at 3.4256184594817367
iteration 4 : approximate location of root at 3.4256184594817283
SUCCESS! Root has been located to within the specified tolerance after 4 iterations.
Root is located at 3.4256184594817283 +/- 8.43769498715119e-15
